# Setup

In [31]:
%pip install OpenAI markdown mdx_truly_sane_lists weasyprint python-dotenv pango

Note: you may need to restart the kernel to use updated packages.


In [32]:
import os

from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

True

Set up the OpenAI Assistant

In [33]:
from openai import OpenAI
client = OpenAI()

#  get the prompt from the file system
with open('prompt.md', 'r') as file:
    # Read the entire content of the file into a single string variable
    prompt = file.read()

# simple retrieve of the Assistant - not necessary because we are doing an update
# my_assistant = client.beta.assistants.retrieve("asst_3WQiw8VJ91XWikievImBqwrZ")

# update the assistant with the latest prompt
my_assistant = client.beta.assistants.update(
  "asst_3WQiw8VJ91XWikievImBqwrZ",
  instructions=prompt,
  name="Interview Playbook Generator",
  tools=[{"type": "retrieval"}],
  model="gpt-4-1106-preview"
  # model="gpt-3.5-turbo-1106"
)

In [34]:
# create a thread
my_thread = client.beta.threads.create()

import time

def create_run(client, my_thread, my_assistant, content, file_ids=None):
    """
    Create a message, run it, and display the response.

    Args:
    client (Client): The API client.
    my_thread (Thread): The thread to post the message to.
    my_assistant (Assistant): The assistant to use for the run.
    content (str): The content of the message.
    file_ids (list, optional): List of file IDs to attach to the message.

    Returns:
    str: The response content.

    Raises:
    ValueError: If any of the parameters are invalid.
    RuntimeError: If the run does not complete successfully.
    """

    if not all([client, my_thread, my_assistant, content]):
        raise ValueError("Invalid parameters provided.")

    extra_instructions = """ ONLY return the content as John Gruber's markdown with no delimiters and with no commentary whatsoever."""

    # Create the message
    thread_message = client.beta.threads.messages.create(
        my_thread.id,
        role="user",
        content=content + extra_instructions,
        file_ids=file_ids or []
    )

    # Hard code the values for max_retries and wait_seconds
    max_retries = 120
    wait_seconds = 0.5

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=my_thread.id,
        assistant_id=my_assistant.id
    )

    # Check the run's status
    for _ in range(max_retries):
        run = client.beta.threads.runs.retrieve(
            thread_id=my_thread.id,
            run_id=run.id)
        if run.status == "completed":
            break
        time.sleep(wait_seconds)
    else:
        raise RuntimeError("Run did not complete successfully.")

    # Retrieve and return the response
    all_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
    return all_messages.data[0].content[0].text.value + "\n\n"


## Initial File Upload and Prompt

In [35]:
import time

# Which example to run
#choice = 'medical-assistant'
#choice = 'infra-engineer'
#choice = 'customer-success-manager'
choice = 'real-estate-litigator'

destination_directory = "examples/" + choice + "/"
html_destination = destination_directory + 'playbook.html'
pdf_destination = destination_directory + 'playbook.pdf'
markdown_destination = destination_directory + 'playbook.md'
word_destination = destination_directory + 'playbook.docx'

company_file = "examples/" + choice + "/about-company.md"

# Upload the company description
file_company_values = client.files.create(
  file=open(company_file, "rb"),
  purpose="assistants"
)

job_description_file = "examples/" + choice + "/job-description.md"
# Open the file and read the contents
with open(job_description_file, 'r') as file:
    # Read the entire content of the file into a single string variable
    job_description = file.read()

# Upload the job description file
file_job_description = client.files.create(
  file=open(job_description_file, "rb"),
  purpose="assistants"
)

# give OpenAI time to process the file
time.sleep(5)


# Priming the Pump

In [36]:
# Refresh the context to ensure it has read the files
primer_values = create_run(client,my_thread,my_assistant,"""
            Write a detailed description of the company values from the about company file, using the company name.
            """, [file_company_values.id])
print(primer_values)

## Nixon Peabody: Mission, Vision, and Values

### Values
Nixon Peabody’s values highlight their commitment to the following principles:

- **Unparalleled Service:** Emphasis on service excellence and ensuring client success, striving to exceed expectations.
  
- **Rich Experience:** Leveraging collective expertise to provide innovative legal solutions.

- **Unshakeable Trust:** Cultivating enduring relationships through trustworthy actions and behaviors.

- **Respectful Inclusion:** Fostering diversity, integrity, and respect within the culture, promoting diverse ideas for effective solutions.

Additionally, Nixon Peabody is dedicated to diversity, equity, and inclusion, sustainability, social responsibility, and technological innovation to enhance client services. These values guide the firm in delivering superior legal services with a strong ethical foundation and a positive societal impact.




In [61]:
# Refresh the context to ensure it has read the files and create context. Not included in final doc.
primer_job = create_run(client,my_thread,my_assistant,"""
            Write the job title contained in the job description file.
            Then, create the list of 5 technical competencies, which MUST cover the areas mentioned in the
            requirements in the job description. Create the list of 5 behavioral
            competencies, with a focus on the company values but still covering all requirements in the
            job description. Where possible, describe a competency in a way that aligns with the company values.
            Each competency should have a title followed by a short description.
            For each competency, list 3 or 4 subcompetencies and their descriptions to enable more deep exploration.
            """, [file_job_description.id])
print(primer_job)


## Real Estate Litigation Department Attorney

### Technical Competencies

#### Commercial and Real Estate Litigation Experience
- **Contractual Dispute Resolution**: Proficiency in handling disputes arising from real estate contracts, including interpretation and enforcement.
- **Property Title Litigation**: Expertise in title issues, easements, and boundary disputes.
- **Construction Law**: Knowledge of construction-related litigation, including defects, delays, and mechanic liens.
- **Land Use and Zoning**: Understanding of land use policies, zoning disputes, and environmental considerations in litigation.

#### Client Service in Fast-Paced Environment
- **Client Communication**: Exceptional skills in client counseling and delivering legal updates promptly.
- **Case Prioritization**: Ability to effectively manage and prioritize a high volume of cases.
- **Strategic Agility**: Skill in rapidly developing and adjusting strategies to meet changing client needs.
- **Crisis Management**:

# Create Introduction

In [62]:
introduction = create_run(client,my_thread,my_assistant,"""
            Generate the Introduction with the Candidate Profle (only if applicable) and Interview Structure. Be sure to include mention of the competencies in the interview structure.
            """)
print(introduction)

# Interview Playbook for Real Estate Litigation Department Attorney

## Introduction

This Interview Playbook is designed to guide the assessment of candidates for the position of Real Estate Litigation Department Attorney. It ensures a structured approach to evaluate their alignment with the key technical and behavioral competencies required for this critical role at the firm. The candidate's expertise in handling complex real estate litigation, their ability to deliver exceptional client service, and their preparedness to work collaboratively in a diverse and fast-paced environment will be the focus of the evaluation.

### Interview Structure

The interview should be broken down into the following sections:

1. **Introduction (5-10 minutes)**
   - Greeting and establishing rapport with the candidate.
   - Providing a brief overview of the firm and the expectations for the Real Estate Litigation Department Attorney position.

2. **Technical Competency Assessment (40-60 minutes)**
   -

# Create Evaluation Criteria

In [63]:
eval_criteria = create_run(client,my_thread,my_assistant,"""
    Generate the Evaluation Critera section.
    """)
print(eval_criteria)

## Evaluation Criteria

The following job scorecard is designed to ensure an objective and comprehensive evaluation of the Real Estate Litigation Department Attorney candidate. It takes into account the specific competencies necessary for the role. Interviewers can score each competency and provide comments on the candidate's performance.

| Competency                               | Assessment | Comments |
|------------------------------------------|------------|----------|
| Commercial and Real Estate Litigation Experience |            |          |
| Client Service in Fast-Paced Environment |            |          |
| California Bar Admission                |            |          |
| Varied Caseload Management              |            |          |
| Complex Legal Issue Resolution          |            |          |
| Client-Centric Focus                    |            |          |
| Collaboration within a Diverse Team     |            |          |
| Adaptive to a Fast-Paced Environ

# Create Technical Assessment

In [65]:
technical_assessment_1 = create_run(client,my_thread,my_assistant,"Generate the first Technical Assessment")
print(technical_assessment_1)


## Technical Assessment

Assessing technical skills allows us to accurately gauge a candidate's legal acumen specifically related to commercial and real estate litigation.

### Technical Competence 1: Commercial and Real Estate Litigation Experience

_Should exhibit a comprehensive understanding of real estate laws and be adept at managing legal processes involved in real estate disputes._

#### Lead Question

"Can you walk us through your most complex commercial or real estate litigation case, detailing the legal strategy you developed and its outcome?"
_Tip: Look for the ability to develop and articulate a clear strategy based on intricate knowledge of real estate law._

#### Probing Questions

- "How do you approach a case with potential conflicts in contractual terms and conditions?"
  _Tip: Assess the candidate’s analytical skills in contract interpretation and dispute resolution._

- "Describe your experience with a property title or boundary dispute and how you resolved it."
  _

In [66]:
technical_assessment_2 = create_run(client,my_thread,my_assistant,"Generate the 2nd Technical Assessment. Don't include the main section header.")
print(technical_assessment_2)

### Technical Competence 2: Client Service in Fast-Paced Environment

_Excellence in servicing clients, particularly in a demanding and fast-paced environment, ensuring that the client's needs are met with efficiency and precision._

#### Lead Question

"Describe a time when you had to quickly adjust your legal strategy to accommodate a client's urgent request. How did you handle the pressure and what was the outcome?"

_Tip: Look for examples that demonstrate the ability to deliver under pressure while maintaining high-quality client service._

#### Probing Questions

- "How do you prioritize and manage a high volume of cases without compromising on the quality of client service?"
  _Tip: Evaluate the candidate’s prioritization and time management skills._

- "Can you share an experience where you needed to rapidly change your approach due to sudden shifts in a case's dynamics?"
  _Tip: Assess strategic agility and the ability to respond effectively to unexpected changes._

- "Discuss

In [67]:
technical_assessment_3 = create_run(client,my_thread,my_assistant,"Generate the 3rd Technical Assessment. Don't include the main section header.")
print(technical_assessment_3)

### Technical Competence 3: California Bar Admission

_Requires an in-depth knowledge of California-specific laws and procedures, crucial for effective practice within the state's jurisdiction._

#### Lead Question

"How do you ensure compliance with California's unique legal landscape in your practice, and could you provide an example where your state-specific knowledge had a significant impact on a case outcome?"

_Tip: Seek detailed examples of how the candidate’s state-specific knowledge positively influenced a case._

#### Probing Questions

- "What are some key considerations you keep in mind when dealing with California-specific real estate laws?"
  _Tip: Evaluate the candidate's understanding of local nuances in real estate law._

- "Describe a time when familiarity with San Francisco's local court procedures gave you an advantage in a case."
  _Tip: Look for instances where procedural know-how led to tactical benefits._

- "How do you stay updated with changes in California li

In [68]:
technical_assessment_4 = create_run(client,my_thread,my_assistant,"Generate the 4th Technical Assessment. Don't include the main section header.")
print(technical_assessment_4)

### Technical Competence 4: Varied Caseload Management

_Effective management of a diverse and extensive caseload is expected, with a focus on organization, time management, delegation, and strategic case review._

#### Lead Question

"Managing a varied caseload requires strong organizational skills. Describe your system for keeping track of different cases and their respective deadlines."

_Tip: Look for evidence of sophisticated organization and time management systems._

#### Probing Questions

- "How do you determine the priority level of cases when everything seems urgent?"
  _Tip: Evaluate the candidate’s ability to prioritize tasks based on urgency and importance._

- "What has been your experience with delegating tasks within a legal team, and how do you ensure quality is maintained?"
  _Tip: Assess the candidate’s approach to teamwork and delegation._

- "Discuss how you conduct a strategic review of your cases and how that influences their outcomes."
  _Tip: Determine the can

In [69]:
technical_assessment_5 = create_run(client,my_thread,my_assistant,"Generate the 5th Technical Assessment. Don't include the main section header.")
print(technical_assessment_5)

### Technical Competence 5: Complex Legal Issue Resolution

_Ability to navigate and resolve intricate legal issues effectively, employing negotiation, research, writing, and advocacy skills._

#### Lead Question

"Describe your process for untangling a complex real estate litigation matter from initial problem identification to resolution."

_Tip: Expect detailed problem-solving methodology and strategic planning._

#### Probing Questions

- "What techniques do you use in negotiation to ensure you achieve the best possible outcome for your client?"
  _Tip: Assess negotiation skills and the ability to attain favorable resolutions._

- "How do you approach legal research when confronted with a unique issue, and what resources do you rely on?"
  _Tip: Look for comprehensive research strategies and resourcefulness._

- "Discuss a written argument you crafted that had a significant impact on the decision in a case."
  _Tip: Evaluate legal writing skills and the ability to persuade through 

In [70]:
technical_assessment = technical_assessment_1 + technical_assessment_2 + technical_assessment_3 + technical_assessment_4 + technical_assessment_5

# Create Behavioral Assessment

In [71]:
behavioral_assessment_1 = create_run(client,my_thread,my_assistant,"Generate the 1st Behavioral Assessment section.")
print(behavioral_assessment_1)

## Behavioral Assessment

Behavioral competencies are as crucial as technical ones, providing insights into the candidate's interpersonal skills and alignment with company culture.

### Behavioral Competence 1: Client-Centric Focus

_Demonstrates a dedication to understanding and fulfilling client needs, ensuring that every action taken is with the client's best interest in mind._

#### Lead Question

"Reflect on a time when you had to make a tough decision that was in the best interest of the client, potentially at the expense of your firm's immediate interests."

_Tip: Look for examples that show the candidate puts client needs first._

#### Probing Questions

- "How do you navigate situations where the client's goals and legal realities do not align?"
  _Tip: Assess how the candidate manages client expectations._

- "Describe a lasting relationship you have developed with a client. What actions do you believe fostered this relationship?"
  _Tip: Seek understanding of how they build 

In [72]:
behavioral_assessment_2 = create_run(client,my_thread,my_assistant,"Generate the 2nd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_2)

### Behavioral Competence 2: Collaboration within a Diverse Team

_Valuing and contributing to teamwork, with an emphasis on respecting diverse perspectives and fostering a collaborative work environment._

#### Lead Question

"Give us an example of a time when you worked as part of a diverse team to achieve a common goal. What did you learn from that experience?"

_Tip: Look for insight into the candidate's ability to work with diverse individuals and learn from different viewpoints._

#### Probing Questions

- "How do you approach a situation where your team members have differing opinions on a case?"
  _Tip: Assess the candidate’s conflict resolution and consensus-building skills._

- "What methods have you employed in the past to ensure clear and constructive team communication?"
  _Tip: Evaluate the mechanisms they use to facilitate effective team communication._

- "Discuss a moment when you took the initiative to support a team member who needed assistance. What prompted you to 

In [73]:
behavioral_assessment_3 = create_run(client,my_thread,my_assistant,"Generate the 3rd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_3)

### Behavioral Competence 3: Adaptive to a Fast-Paced Environment

_Thrives in dynamic, high-pressure situations, demonstrating resilience, quick decision-making, and effective multitasking capabilities._

#### Lead Question

"Can you discuss a time when you had to quickly adapt to a fast-paced change in your work environment? How did you maintain your effectiveness?"

_Tip: Observe the candidate's ability to maintain composure and efficiency under rapid change._

#### Probing Questions

- "How do you manage stress in a high-pressure situation without it affecting your performance?"
  _Tip: Gauge the candidate's stress management strategies and resilience._

- "Describe a situation where you had to make a critical decision quickly. What was the situation and how did you approach it?"
  _Tip: Assess decision-making skills under time constraints._

- "Tell me about a time when you had to juggle multiple responsibilities at once. How did you ensure all tasks were completed successfully?"


In [74]:
behavioral_assessment_4 = create_run(client,my_thread,my_assistant,"Generate the 4th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_4)

### Behavioral Competence 4: Ethical Practice and Responsibility

_Dedicated to upholding the highest standards of integrity and responsible conduct in all professional matters, ensuring the trust of clients and the respect of peers._

#### Lead Question

"Provide an example of a difficult ethical dilemma you have faced in your career and how you resolved it."

_Tip: Look for a clear ethical stance and appropriate resolution actions._

#### Probing Questions

- "How do you maintain professional responsibility, particularly when under tight deadlines or client pressure?"
  _Tip: Assess the candidate’s ability to withstand pressure without compromising ethical standards._

- "Describe how you ensure confidentiality in sensitive client situations."
  _Tip: Evaluate the candidate’s procedures for protecting privileged information._

- "Tell us about a time when you identified a regulatory compliance issue and the steps you took to correct it."
  _Tip: Determine the candidate’s proactivenes

In [75]:
behavioral_assessment_5 = create_run(client,my_thread,my_assistant,"Generate the 5th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_5)

### Behavioral Competence 5: Continuous Learning and Skill Development

_Demonstrates a commitment to personal and professional growth, actively keeping abreast of legal trends, and utilizing feedback to enhance performance._

#### Lead Question

"Describe a situation where you had to learn a new area of law quickly to assist a client. How did you approach this learning challenge?"

_Tip: Evaluate the candidate's learning agility and dedication to professional development._

#### Probing Questions

- "How do you stay current with the latest developments in real estate litigation?"
  _Tip: Assess methods for ongoing education and staying informed._

- "Can you give an example of a time when you received constructive feedback? How did you respond and what was the result?"
  _Tip: Gauge receptivity to feedback and the ability to use it for improvement._

- "Share your experience with implementing a new technology or process within your legal practice."
  _Tip: Look for adaptability and in

In [76]:
behavioral_assessment = behavioral_assessment_1 + behavioral_assessment_2 + behavioral_assessment_3 + behavioral_assessment_4 + behavioral_assessment_5

# Create Practical Assignment

In [77]:
practical_assignment = create_run(client,my_thread,my_assistant,"Generate the Practical Assignment section.")
print(practical_assignment)


## Practical Assignment

The practical assignment for the Real Estate Litigation Department Attorney role will evaluate the candidate's abilities to apply their legal knowledge and skills in a real-world context.

**Assignment:** The candidate will be given a hypothetical but realistic case scenario involving a complex real estate dispute. They will be tasked with drafting a motion to dismiss, including legal arguments and supporting evidence.

**Objectives:**

- To assess the candidate’s legal analytical skills.
- To evaluate their ability to write clear, persuasive legal documents.
- To judge their strategic thinking in formulating arguments.

**Evaluation Criteria:**

- **Clarity of Argument**: The motion should present a clear and logical argument.
- **Persuasiveness**: It should be convincing, with well-supported claims.
- **Knowledge of Law**: The motion must demonstrate a deep understanding of applicable laws and precedents.
- **Attention to Detail**: The candidate should show a

# Create Team Interaction

In [78]:
team_interaction = create_run(client,my_thread,my_assistant,"Generate the Team Interaction section.")
print(team_interaction)

## Team Fit

Evaluating how a candidate interacts with the team provides critical insights into their potential fit within the firm's culture and work environment.

**Discussion Points:**

- The candidate's ability to engage in a collaborative work style.
- Openness to different perspectives and constructive feedback.
- Willingness to share knowledge and contribute to the team's success.
- Approach to conflict resolution within a team setting.

**Activities:**

- Collaborative problem-solving exercises related to real estate litigation.
- Group discussions on hypothetical cases or recent legal developments.
- Simulations of team meetings to plan strategy for complex cases.

Observing the candidate in these interactions will allow for an assessment of their communication skills, adaptability, and compatibility with the team and overall firm culture.




# Create Candidate Q&A

In [79]:
candidate_questions = create_run(client,my_thread,my_assistant,"""
    Generate the Candidate Q&A section. Include example answers.
    """)
print(candidate_questions)

## Candidate Q&A

Candidates often have questions about the role, the firm, and the expectations. Here are six potential questions with comprehensive answers that reflect the firm’s practices and culture.

**Question:** "What opportunities for professional growth does the firm offer?"

**Answer:** "We have a robust professional development program that includes mentorship, ongoing legal education, and opportunities to work on diverse and challenging cases."

**Question:** "How does the firm ensure work-life balance for its attorneys?"

**Answer:** "We encourage work-life balance through flexible work arrangements, supportive team structures, and an understanding that personal well-being is crucial to professional success."

**Question:** "Can you tell me more about the firm’s culture and how new attorneys are integrated into the team?"

**Answer:** "Our firm prides itself on a collegial atmosphere where new attorneys are welcomed with mentorship opportunities and inclusive team project

# Finalize the Playbook HTML

## Create a markdown version

For use in GitHub as an example

In [80]:
playbook_md = (introduction + "\n\n" +
            job_description + "\n\n" +
            eval_criteria + "\n\n" +
            technical_assessment + "\n\n" +
            behavioral_assessment + "\n\n" +
            practical_assignment + "\n\n" +
            candidate_questions + "\n\n" +
            team_interaction + "\n\n" )

# Write playbook to a text file
with open(markdown_destination, 'w') as file:
    file.write(playbook_md)

## Convert the markdown to HTML

We are doing this in chunks so that we can do more useful things with it in the HTML output

In [81]:
import markdown

# Convert Markdown to HTML
html_playbook = markdown.markdown(playbook_md, extensions=['extra','mdx_truly_sane_lists'])

# Convert Markdown to HTML
html_job_description = markdown.markdown(job_description, extensions=['extra','mdx_truly_sane_lists'])
html_introduction = markdown.markdown(introduction, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment = markdown.markdown(technical_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment = markdown.markdown(behavioral_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_practical_assignment = markdown.markdown(practical_assignment, extensions=['extra','mdx_truly_sane_lists'])
html_candidate_questions = markdown.markdown(candidate_questions, extensions=['extra','mdx_truly_sane_lists'])
html_team_interaction = markdown.markdown(team_interaction, extensions=['extra','mdx_truly_sane_lists'])
html_eval_criteria = markdown.markdown(eval_criteria, extensions=['extra','mdx_truly_sane_lists'])


html_technical_assessment_1 = markdown.markdown(technical_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_2 = markdown.markdown(technical_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_3 = markdown.markdown(technical_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_4 = markdown.markdown(technical_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_5 = markdown.markdown(technical_assessment_5, extensions=['extra','mdx_truly_sane_lists'])

html_behavioral_assessment_1 = markdown.markdown(behavioral_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_2 = markdown.markdown(behavioral_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_3 = markdown.markdown(behavioral_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_4 = markdown.markdown(behavioral_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_5 = markdown.markdown(behavioral_assessment_5, extensions=['extra','mdx_truly_sane_lists'])



## Build the HTML

Borrowed colors from this [color palette](https://color.adobe.com/PALETTE-mini-06:-Transparent-color-theme-15d8c4a6-b06a-4530-bf5b-3fc8102ccd69)

In [82]:
# Open the file and read the contents
with open('styles.css', 'r') as file:
    # Read the entire content of the file into a single string variable
    styles = file.read()


html_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

html_short_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div class="short-dotted-line"></div>
    </div>
</div>"""

from datetime import datetime

# Getting today's date in a nicely formatted string for US English
publication_date = datetime.now().strftime("%B %d, %Y")


# Define the HTML template
html_content = rf"""
<meta charset="utf-8">
<html>
<head>
  <link href="https://fonts.googleapis.com/css2?family=Source%20Sans%20Pro" rel="stylesheet">
  <link href="https://fonts.googleapis.com/css2?family=Libre%20Baskerville" rel="stylesheet">
  <style>
  {styles}
  </style>
</head>
<body>
    <section class="intro" id="intro">
    <div class="publication-date">Published on {publication_date} by the <a href="https://github.com/androb/deliverables/tree/main/interviewplaybook">Interview Playbook Generator</a>.</div>
    {html_introduction}
    </section>
    <section class='main' id="contents">
      <h1>Table of Contents</h1>
      <ul>
        <li><a href="#job_description">Job Description</a></li>
        <li><a href="#eval_criteria">Evaluation Criteria</a></li>
        <li><a href="#technical_assessment_1">Technical Assessment</a></li>
        <li><a href="#behavioral_assessment_1">Behavioral Assessment</a></li>
        <li><a href="#practical_assignment">Practical Assignment</a></li>
        <li><a href="#team_interaction">Team Interaction</a></li>
        <li><a href="#candidate_questions">Candidate Questions</a></li>
      </ul>
    </section>
    <section class='main' id="job_description">
    {html_job_description}
    </section>
    <section class='main' id="eval_criteria">
    {html_eval_criteria}
    </section>
    <section class='main' id="technical_assessment_1">
      {html_technical_assessment_1}
     {html_short_notes}
    </section>
    <section class='main' id="technical_assessment_2">
      {html_technical_assessment_2}
     {html_notes}
    </section>
        <section class='main' id="technical_assessment_3">
      {html_technical_assessment_3}
     {html_notes}
    </section>
    <section class='main' id="technical_assessment_4">
      {html_technical_assessment_4}
     {html_notes}
    </section>
        <section class='main' id="technical_assessment_5">
      {html_technical_assessment_5}
     {html_notes}
    </section>
    <section class="main" id="behavioral_assessment_1">
    {html_behavioral_assessment_1}
     {html_short_notes}
    </section>
    <section class="main" id="behavioral_assessment_2">
    {html_behavioral_assessment_2}
     {html_notes}
    </section>
    <section class="main" id="behavioral_assessment_3">
    {html_behavioral_assessment_3}
     {html_notes}
    </section>
        <section class="main" id="behavioral_assessment_4">
    {html_behavioral_assessment_4}
     {html_notes}
    </section>
        <section class="main" id="behavioral_assessment_5">
    {html_behavioral_assessment_5}
     {html_notes}
    </section>
    <section class="main" id="practical_assignment">
    {html_practical_assignment}
    {html_notes}
    </section>
    <section class="main" id="team_interaction">
    {html_team_interaction}
    {html_notes}
    </section>
    <section class="main" id="candidate_questions">
    {html_candidate_questions}
    </section>

</body>
</html>"""

# Write HTML string to a file
with open(html_destination, 'w') as file:
    file.write(html_content)



Display the HTML content here so we can review it

In [83]:
# Display the Markdown-formatted text in the notebook
from IPython.display import display, HTML
# display(HTML(html_content))

# Create the Documents

Create the PDF

In [84]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
#HTML('playbook.html').write_pdf('playbook.pdf', stylesheets=['style.css'], presentational_hints=True)
HTML(html_destination).write_pdf(pdf_destination, presentational_hints=True)



Create the Word document

In [85]:
# Convert HTML to Docx
print(html_destination)
print(word_destination)
!pandoc -s html_destination -o word_destination

# from google.colab import files
# files.download('playbook.docx')

examples/real-estate-litigator/playbook.html
examples/real-estate-litigator/playbook.docx
[WARNING] Could not deduce format from file extension 
  Defaulting to markdown
[WARNING] Could not deduce format from file extension 
  Defaulting to html
pandoc: html_destination: withBinaryFile: does not exist (No such file or directory)
